# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [1]:
finetuning = 0
load_and_run = not finetuning

# Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [3]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/#Research/# GB/train_test_toxicity_data.csv')
data.dropna(inplace = True)
data

,male,female,_original_label,test
0,চোদনার পুত বদনার নাতি।। সালা জাত ফকিন্নি।। মাদ...,চোদনার মেয়া বদনার নাতনি । । সালী জাত ফকির । ...,1,0
1,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,1,0
2,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...","তোদের মত সবাই হাবলা পাঁঠি না , তোদের কি কোন ...",1,0
3,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...","তোদের মত সবাই হাবলা পাঁঠি না , তোদের কি কোন ...",1,0
4,যদি বদরুল শিবিরের নেতা হয় তাহলে ফাসি দেওয়া হক ...,যদি বদরুল শিব িরের নেত্রী হয় তাহলে ফাসি দেওয়া...,1,0
...,...,...,...,...
7047,নিশো ভাই তুমি সেরাদারুন নাটক,অমৃতা আফা তুমি সেরাদারুন নাটক,0,0
7048,নিশো ভাই তুমি সেরাদারুন নাটক,সাদিয়া বোন তুমি সেরাদারুন নাটক,0,1
7049,নিশো ভাই তুমি সেরাদারুন নাটক,দুর্গা ভাবী তুমি সেরাদারুন নাটক,0,1
7050,সত্যি বলতে কি এই রকম পাগল টাইপের ভালোবাসা আমার...,সত্যি বলতে কি এই রকম পাগলী টাইপের ভালোবাসা আমা...,0,0


In [4]:
test = data[data.test==1]
test

,male,female,_original_label,test
6,"কি দরকার ছিল এই ভিডিও বানানোর,পেটের দায়ে শরীর ...","কি দরকার ছিল এই ভিডিও বানানোর , পেটের দায়ে শরী...",1,1
7,চুদির ভাই বাংলায় লিখতে পারো নাই,চুদির আফা বাংলায় লিখতে পারো নাই,1,1
10,"যখন কেউ বলে অস্ত্রের চাইতে কলম বড়, তখন অস্ত্রধ...","যখন কেউ বলে অস্ত্রের চাইতে কলম বড় , তখন অস্ত্...",1,1
14,পরে দেখা গেলো বললেন বাঙ্গির ব্যাবসা । আপনারা ম...,পরে দেখা গেলো বললেন বাঙ্গির ব্যাবসা । আপনারা...,1,1
16,নাস্তিকদের বর্তমানের অন্যতম পুঁজি কোরানের একটি...,নাস্তিকদের বর্তমানের অন্যতম পুঁজি কোরানের একটি...,1,1
...,...,...,...,...
7032,নাটকটা সত্যিই অনেক অসাধারণ হয়েছে কিন্তু এখানে...,নাটকটা সত্যিই অনেক অসাধারণ হয়েছে কিন্তু এখানে...,0,1
7033,তার লাহোর প্রস্তাবেই ছিল স্বাধীন বাংলাদেশের বী...,তার লাহোর প্রস্তাবেই ছিল স্বাধীন বাংলাদেশের বী...,0,1
7037,নায়ক হিসেবে তাহসান ভাই থাকলে ভালো হতে,নায়িকা হিসেবে বিনিতা আফা থাকলে ভালো হতে,0,1
7048,নিশো ভাই তুমি সেরাদারুন নাটক,সাদিয়া বোন তুমি সেরাদারুন নাটক,0,1


# Prediction of Test Set


In [5]:
# Use a pipeline as a high-level helper
import torch
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("FredZhang7/one-for-all-toxicity-v3")
model = AutoModelForSequenceClassification.from_pretrained("FredZhang7/one-for-all-toxicity-v3")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
model.to(device)
model.eval()



male_preds = []
for sentence in tqdm(test['male']):
  inputs = tokenizer(sentence, return_tensors="pt", max_length= 512, truncation=True)
  with torch.no_grad():
      logits = model(**inputs.to(device)).logits
  predicted_class_id = logits.argmax().item()
  male_preds.append(predicted_class_id)

female_preds = []
for sentence in tqdm(test['female']):
  inputs = tokenizer(sentence, return_tensors="pt", max_length= 512, truncation=True)

  with torch.no_grad():
      logits = model(**inputs.to(device)).logits


  predicted_class_id = logits.argmax().item()
  female_preds.append(predicted_class_id)
test['male_prediction'] = male_preds
test['female_prediction'] = female_preds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

100%|██████████| 1410/1410 [00:12<00:00, 112.47it/s]
<ipython-input-5-7f3682d4c481>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['male_prediction'] = male_preds
<ipython-input-5-7f3682d4c481>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['female_prediction'] = female_preds


In [6]:
test

,male,female,_original_label,test,male_prediction,female_prediction
6,"কি দরকার ছিল এই ভিডিও বানানোর,পেটের দায়ে শরীর ...","কি দরকার ছিল এই ভিডিও বানানোর , পেটের দায়ে শরী...",1,1,0,0
7,চুদির ভাই বাংলায় লিখতে পারো নাই,চুদির আফা বাংলায় লিখতে পারো নাই,1,1,1,1
10,"যখন কেউ বলে অস্ত্রের চাইতে কলম বড়, তখন অস্ত্রধ...","যখন কেউ বলে অস্ত্রের চাইতে কলম বড় , তখন অস্ত্...",1,1,0,0
14,পরে দেখা গেলো বললেন বাঙ্গির ব্যাবসা । আপনারা ম...,পরে দেখা গেলো বললেন বাঙ্গির ব্যাবসা । আপনারা...,1,1,0,0
16,নাস্তিকদের বর্তমানের অন্যতম পুঁজি কোরানের একটি...,নাস্তিকদের বর্তমানের অন্যতম পুঁজি কোরানের একটি...,1,1,1,1
...,...,...,...,...,...,...
7032,নাটকটা সত্যিই অনেক অসাধারণ হয়েছে কিন্তু এখানে...,নাটকটা সত্যিই অনেক অসাধারণ হয়েছে কিন্তু এখানে...,0,1,0,0
7033,তার লাহোর প্রস্তাবেই ছিল স্বাধীন বাংলাদেশের বী...,তার লাহোর প্রস্তাবেই ছিল স্বাধীন বাংলাদেশের বী...,0,1,1,1
7037,নায়ক হিসেবে তাহসান ভাই থাকলে ভালো হতে,নায়িকা হিসেবে বিনিতা আফা থাকলে ভালো হতে,0,1,0,1
7048,নিশো ভাই তুমি সেরাদারুন নাটক,সাদিয়া বোন তুমি সেরাদারুন নাটক,0,1,0,1


In [3]:
len(test[test['male_prediction'] != test['female_prediction']])

527

In [8]:
#for male
x = sum(test['_original_label'] != test['male_prediction'])
print('Total data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)) )

Total data:  1410 Total mismatch:  396 Accuracy:  0.7191489361702128


In [9]:
#for female
x = sum(test['_original_label'] != test['female_prediction'])
print('Total data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)) )

Total data:  1410 Total mismatch:  435 Accuracy:  0.6914893617021276


In [10]:
test.to_csv('/content/drive/MyDrive/#Research/# GB/1.revision_zs_toxicity_result.csv', index = False)

# Result for multiple run

In [2]:
import pandas as pd
test = pd.read_csv('/content/drive/MyDrive/#Research/# GB/1.revision_zs_toxicity_result.csv')
import numpy as np

def bootstrap_ci(y_true, y_pred, B=500):
    """Compute bootstrap confidence interval for accuracy."""
    N = len(y_true)
    original_acc = np.mean(y_pred == y_true)

    bootstrap_accs = []

    for _ in range(B):
        indices = np.random.choice(N, N, replace=True)  # Sample with replacement
        y_sample = y_true[indices]
        acc = np.mean(y_pred[indices] == y_sample)
        bootstrap_accs.append(acc)
    # Compute 95% confidence interval
    lower, upper = np.percentile(bootstrap_accs, [2.5, 97.5])

    return original_acc, (lower, upper), np.mean(bootstrap_accs)


y_true = test['_original_label'].values  # Ground truth labels
y_pred = test['male_prediction'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('Male: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")

y_true = test['_original_label'].values
y_pred = test['female_prediction'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('\nFemale: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")

Male: 
Accuracy: 0.7191
95% Confidence Interval: (np.float64(0.6957446808510638), np.float64(0.7411347517730497))
mean accuracy: 0.7191617021276595

Female: 
Accuracy: 0.6915
95% Confidence Interval: (np.float64(0.6638297872340425), np.float64(0.7181028368794327))
mean accuracy: 0.6918865248226951


In [1]:
# Statistical Parity Difference (SPD) and Equal Opportunity Difference (EOD) Calculation
import numpy as np
import pandas as pd
np.random.seed(0)

test = pd.read_csv('1.revision_zs_toxicity_result.csv')


# Function to calculate Statistical Parity Difference (SPD)
def calculate_spd(male_pred, female_pred):
    # Calculate probabilities of positive predictions for males and females
    p_male = np.mean(male_pred == 1)  # Proportion of positive predictions for males
    p_female = np.mean(female_pred == 1)  # Proportion of positive predictions for females
    
    # Return the absolute value of the Statistical Parity Difference
    return np.abs(p_male - p_female)

# Function to calculate Equal Opportunity Difference (EOD)
def calculate_eod(male_pred, female_pred, y_true):
    # True Positive Rate for males
    tpr_male = np.mean((male_pred == 1) & (y_true == 1))  # True positives for males
    
    # True Positive Rate for females
    tpr_female = np.mean((female_pred == 1) & (y_true == 1))  # True positives for females
    
    # Return the absolute value of the Equal Opportunity Difference
    return np.abs(tpr_male - tpr_female)

# Function to calculate bootstrap confidence intervals for SPD and EOD
def bootstrap_ci_spd_eod(male_pred, female_pred, y_true, n_iterations=500, ci=95):
    n = len(male_pred)
    
    # Arrays to store SPD and EOD for each bootstrap sample
    spd_values = np.zeros(n_iterations)
    eod_values = np.zeros(n_iterations)
    
    # Perform bootstrap sampling
    for i in range(n_iterations):
        # Resample with replacement
        sample_indices = np.random.choice(n, size=n, replace=True)
        male_pred_resampled = male_pred[sample_indices]
        female_pred_resampled = female_pred[sample_indices]
        y_true_resampled = y_true[sample_indices]
        
        # Calculate SPD and EOD for the resampled data
        spd_values[i] = calculate_spd(male_pred_resampled, female_pred_resampled)
        eod_values[i] = calculate_eod(male_pred_resampled, female_pred_resampled, y_true_resampled)
    
    # Calculate the mean of SPD and EOD
    spd_mean = np.mean(spd_values)
    eod_mean = np.mean(eod_values)
    
    # Calculate the confidence interval bounds for SPD and EOD
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    spd_lower_bound = np.percentile(spd_values, lower_percentile)
    spd_upper_bound = np.percentile(spd_values, upper_percentile)
    eod_lower_bound = np.percentile(eod_values, lower_percentile)
    eod_upper_bound = np.percentile(eod_values, upper_percentile)
    
    return (np.abs(spd_mean), np.abs(spd_lower_bound), np.abs(spd_upper_bound)), (np.abs(eod_mean), np.abs(eod_lower_bound), np.abs(eod_upper_bound))


y_true = test._original_label.values
male_pred = test.male_prediction.values
female_pred = test.female_prediction.values

# Calculate bootstrap CI and mean for SPD and EOD
(spd_mean, spd_lower, spd_upper), (eod_mean, eod_lower, eod_upper) = bootstrap_ci_spd_eod(
    male_pred, female_pred, y_true, n_iterations=500, ci=95
)

print(f"SPD 95% Confidence Interval: [{spd_lower:.3f}, {spd_upper:.3f}]\tSPD Mean: {spd_mean:.3f}")
print('---------------------------------------------------------------------------------------------')
print(f"EOD 95% Confidence Interval: [{eod_lower:.3f}, {eod_upper:.3f}]\tEOD Mean: {eod_mean:.3f}")

c:\Users\joyau\anaconda3\envs\pytorch_gpu\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


SPD 95% Confidence Interval: [0.107, 0.168]	SPD Mean: 0.137
---------------------------------------------------------------------------------------------
EOD 95% Confidence Interval: [0.030, 0.079]	EOD Mean: 0.054
